In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from category_encoders import TargetEncoder
import xgboost as xgb
import lightgbm as lgb
import warnings
import joblib
import os

warnings.filterwarnings('ignore')
np.random.seed(42)

def main():
    print("Loading and preparing data...")
    # Load data
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    
    # Initial exploration
    print(f"Training data shape: {train_df.shape}")
    print(f"Testing data shape: {test_df.shape}")
    
    # Get all possible state values from both datasets
    all_states = set(train_df['job_state'].dropna().unique()).union(set(test_df['job_state'].dropna().unique()))
    all_feature1 = set(train_df['feature_1'].dropna().unique()).union(set(test_df['feature_1'].dropna().unique()))
    
    # Process training data
    X_train, y_train, categorical_cols = preprocess_data(train_df, all_states, all_feature1, is_training=True)
    print(f"Processed training data shape: {X_train.shape}")
    
    # Process test data (using same transformers as training)
    X_test, _, _ = preprocess_data(test_df, all_states, all_feature1, is_training=False, categorical_cols=categorical_cols)
    print(f"Processed test data shape: {X_test.shape}")
    
    # Verify columns match
    missing_cols = set(X_train.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0
    
    # Ensure columns are in the same order
    X_test = X_test[X_train.columns]
    
    # Train multiple models and create ensemble
    best_model = build_and_train_models(X_train, y_train)
    
    # Make predictions
    predictions = best_model.predict(X_test)
    
    # Map numeric predictions back to original labels
    le = LabelEncoder()
    le.fit(train_df['salary_category'])
    predictions_labels = le.inverse_transform(predictions)
    
    # Create submission file
    submission = pd.DataFrame({
        'obs': test_df['obs'],
        'salary_category': predictions_labels
    })
    
    submission.to_csv('solution_format.csv', index=False)
    print("Predictions saved to solution_format.csv")
    print(f"Prediction distribution: {pd.Series(predictions_labels).value_counts().to_dict()}")

def preprocess_data(df, all_states, all_feature1, is_training=True, categorical_cols=None):
    # Define feature groups
    categorical_features = ['job_title', 'job_state', 'feature_1']
    boolean_features = ['feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_10', 'feature_11']
    numerical_features = ['feature_2', 'feature_9', 'feature_12'] + [f'job_desc_{str(i).zfill(3)}' for i in range(1, 301)]
    date_feature = ['job_posted_date']
    
    # Make a copy of the dataframe to avoid modifying the original
    data = df.copy()
    
    # Store the target if it exists
    if 'salary_category' in data.columns and is_training:
        target = data['salary_category']
        # Label encode target for model training
        le = LabelEncoder()
        y = le.fit_transform(target)
    elif is_training:
        raise ValueError("Training data doesn't contain target column 'salary_category'")
    else:
        y = None

    # Handle missing values in numerical features
    for col in numerical_features:
        if col in data.columns:
            if data[col].dtype == 'object':
                data[col] = data[col].replace('', np.nan)
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Convert boolean features to 0/1
    for col in boolean_features:
        if col in data.columns:
            data[col] = data[col].fillna(False).astype(int)
    
    # Handle job_title
    if 'job_title' in data.columns:
        data['job_title'] = data['job_title'].fillna('Unknown')
        
        # Create new feature based on job title patterns
        data['is_senior'] = data['job_title'].str.lower().str.contains('senior|sr|lead|principal').fillna(False).astype(int)
        data['is_junior'] = data['job_title'].str.lower().str.contains('junior|jr|associate|entry').fillna(False).astype(int)
        data['is_developer'] = data['job_title'].str.lower().str.contains('develop|programmer|coder|engineer').fillna(False).astype(int)
        data['is_specialist'] = data['job_title'].str.lower().str.contains('special|expert|consult').fillna(False).astype(int)
        
        # Group rare job titles
        title_counts = data['job_title'].value_counts()
        rare_titles = title_counts[title_counts < 10].index
        data['job_title'] = data['job_title'].apply(lambda x: 'Other_Title' if x in rare_titles else x)
        
        if is_training:
            # Save encoder for test data
            job_encoder = TargetEncoder()
            data['job_title_encoded'] = job_encoder.fit_transform(data['job_title'], target)
            joblib.dump(job_encoder, 'job_title_encoder.joblib')
        else:
            # Load encoder for test data
            if os.path.exists('job_title_encoder.joblib'):
                job_encoder = joblib.load('job_title_encoder.joblib')
                data['job_title_encoded'] = job_encoder.transform(data['job_title'])
            else:
                # If no encoder found, use mean encoding as fallback
                data['job_title_encoded'] = 0.5  # Default value

    # Handle state and feature_1 - ensure consistent one-hot encoding across train and test
    for col in ['job_state', 'feature_1']:
        if col in data.columns:
            data[col] = data[col].fillna('Unknown')
    
    # Process date feature with robust error handling
    if 'job_posted_date' in data.columns:
        data['job_posted_date'] = data['job_posted_date'].fillna('2000/01')
        
        # Extract year and month safely
        def extract_year(date_str):
            try:
                return int(date_str[:4])
            except (TypeError, ValueError, IndexError):
                return 2000
                
        def extract_month(date_str):
            try:
                return int(date_str.split('/')[1])
            except (TypeError, ValueError, IndexError):
                return 1
                
        data['job_posted_year'] = data['job_posted_date'].apply(extract_year)
        data['job_posted_month'] = data['job_posted_date'].apply(extract_month)
        
        # Create cyclical features for month to capture seasonality
        data['month_sin'] = np.sin(2 * np.pi * data['job_posted_month']/12)
        data['month_cos'] = np.cos(2 * np.pi * data['job_posted_month']/12)
        
        # Create feature for recency (assuming more recent jobs might have different patterns)
        data['job_recency'] = data['job_posted_year'] * 12 + data['job_posted_month']
        
        # Handle year ranges - normalize to a reasonable range
        mean_year = 2022
        data['job_posted_year_norm'] = data['job_posted_year'] - mean_year
    
    # Handle feature_9 (important from your feature importance)
    if 'feature_9' in data.columns:
        # Fill missing with median
        median_val = data['feature_9'].median() if not pd.isna(data['feature_9'].median()) else 0
        data['feature_9'] = data['feature_9'].fillna(median_val)
        
        # Create bins for feature_9 - handle edge cases to avoid errors
        try:
            data['feature_9_bin'] = pd.qcut(data['feature_9'].rank(method='first'), 
                                            q=5, 
                                            labels=[0, 1, 2, 3, 4]).astype(int)
        except ValueError:
            # Fallback if qcut fails
            data['feature_9_bin'] = pd.cut(data['feature_9'], 
                                          bins=5, 
                                          labels=[0, 1, 2, 3, 4],
                                          include_lowest=True).astype(int)
        
        # Interaction features
        if 'feature_2' in data.columns:
            data['feature_2_9_interaction'] = data['feature_2'] * data['feature_9']
    
    # Feature_2 transformations (most important feature)
    if 'feature_2' in data.columns:
        data['feature_2'] = data['feature_2'].fillna(data['feature_2'].median())
        data['feature_2_squared'] = data['feature_2'] ** 2
        data['feature_2_sqrt'] = np.sqrt(np.abs(data['feature_2']))
        try:
            data['feature_2_bin'] = pd.qcut(data['feature_2'].rank(method='first'), 
                                            q=5, 
                                            labels=[0, 1, 2, 3, 4]).astype(int)
        except ValueError:
            # Fallback if qcut fails
            data['feature_2_bin'] = pd.cut(data['feature_2'], 
                                          bins=5, 
                                          labels=[0, 1, 2, 3, 4],
                                          include_lowest=True).astype(int)
    
    # Sum of boolean features (second most important feature)
    boolean_cols = [col for col in boolean_features if col in data.columns]
    if boolean_cols:
        data['boolean_sum'] = data[boolean_cols].sum(axis=1)
        data['boolean_sum_squared'] = data['boolean_sum'] ** 2
    else:
        data['boolean_sum'] = 0
        data['boolean_sum_squared'] = 0
    
    # Feature 10 was important
    if 'feature_10' in data.columns and 'feature_8' in data.columns:
        data['feature_10_8_interaction'] = data['feature_10'] * data['feature_8']
    
    # Create aggregations of job_desc features
    job_desc_cols = [f'job_desc_{str(i).zfill(3)}' for i in range(1, 301) if f'job_desc_{str(i).zfill(3)}' in data.columns]
    if job_desc_cols:
        # Fill NAs with 0 for job desc columns
        data[job_desc_cols] = data[job_desc_cols].fillna(0)
        
        # Create aggregate features
        data['job_desc_mean'] = data[job_desc_cols].mean(axis=1)
        data['job_desc_std'] = data[job_desc_cols].std(axis=1)
        data['job_desc_min'] = data[job_desc_cols].min(axis=1)
        data['job_desc_max'] = data[job_desc_cols].max(axis=1)
        data['job_desc_sum'] = data[job_desc_cols].sum(axis=1)
        
        # Create percentile features - capture distribution properties
        data['job_desc_q25'] = data[job_desc_cols].quantile(0.25, axis=1)
        data['job_desc_q75'] = data[job_desc_cols].quantile(0.75, axis=1)
        data['job_desc_iqr'] = data['job_desc_q75'] - data['job_desc_q25']
        
        # Reduced dimensionality of job_desc features using PCA
        if len(job_desc_cols) > 10:  # Only if we have enough features
            if is_training:
                # Fit PCA on training data
                pca = PCA(n_components=15)  # Keep top 15 components
                job_desc_pca = pca.fit_transform(data[job_desc_cols])
                # Save PCA model
                joblib.dump(pca, 'job_desc_pca.joblib')
            else:
                # Use pre-trained PCA for test data
                if os.path.exists('job_desc_pca.joblib'):
                    pca = joblib.load('job_desc_pca.joblib')
                    job_desc_pca = pca.transform(data[job_desc_cols])
                else:
                    # Fallback if no PCA model found
                    job_desc_pca = np.zeros((data.shape[0], 15))
            
            # Add PCA features
            for i in range(min(15, job_desc_pca.shape[1])):
                data[f'job_desc_pca_{i}'] = job_desc_pca[:, i]
    
    # Manual one-hot encoding for job_state to ensure consistent columns
    # This is the key fix for your feature mismatch error
    for state in all_states:
        data[f'state_{state}'] = (data['job_state'] == state).astype(int)
    
    # Manual one-hot encoding for feature_1
    for feat in all_feature1:
        data[f'feat1_{feat}'] = (data['feature_1'] == feat).astype(int)

    # Drop original columns after one-hot encoding
    data = data.drop(['job_state', 'feature_1'], axis=1, errors='ignore')
    
    # Create feature set for modeling
    columns_to_exclude = ['obs', 'job_title', 'salary_category', 'job_posted_date'] + job_desc_cols
    feature_cols = [col for col in data.columns if col not in columns_to_exclude]
    
    # Drop any constant columns
    for col in list(feature_cols):  # Create a copy of the list to avoid modification during iteration
        if col in data.columns and col in feature_cols:
            if len(data[col].unique()) <= 1:
                feature_cols.remove(col)
    
    X = data[feature_cols]
    
    if is_training:
        # Save the column order for test data
        joblib.dump(feature_cols, 'feature_columns.joblib')
        return X, y, feature_cols
    else:
        if categorical_cols is not None:
            # Ensure all training columns exist in test data
            for col in categorical_cols:
                if col not in X.columns:
                    X[col] = 0
            return X, y, categorical_cols
        else:
            # Load columns if not provided
            if os.path.exists('feature_columns.joblib'):
                categorical_cols = joblib.load('feature_columns.joblib')
                # Add missing columns
                for col in categorical_cols:
                    if col not in X.columns:
                        X[col] = 0
            return X, y, categorical_cols

def build_and_train_models(X, y):
    # Advanced hyperparameters for better performance
    # Split data for initial validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    print("Training multiple models...")
    
    # Base models with better hyperparameters
    rf = RandomForestClassifier(
        n_estimators=300, 
        max_depth=None,  # Allow deeper trees 
        min_samples_leaf=1,
        min_samples_split=2,
        max_features='sqrt',
        bootstrap=True,
        class_weight='balanced',
        random_state=42
    )
    
    gb = GradientBoostingClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,  # Slower learning rate for better generalization
        subsample=0.8,
        random_state=42
    )
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=300, 
        learning_rate=0.05,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=3,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1,
        scale_pos_weight=1,
        random_state=42
    )
    
    lgb_model = lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=8,
        num_leaves=40,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1,
        min_child_samples=20,
        random_state=42
    )
    
    # Feature selection with a lower threshold to keep more features
    print("Performing feature selection...")
    selector = SelectFromModel(
        RandomForestClassifier(n_estimators=200, random_state=42),
        threshold='mean'  # Less aggressive threshold
    )
    selector.fit(X_train, y_train)
    selected_features = X_train.columns[selector.get_support()]
    
    print(f"Selected {len(selected_features)} features out of {X_train.shape[1]}")
    
    # Train models on selected features
    X_train_selected = selector.transform(X_train)
    X_val_selected = selector.transform(X_val)
    
    # Evaluate base models
    models = {
        'Random Forest': rf,
        'Gradient Boosting': gb,
        'XGBoost': xgb_model,
        'LightGBM': lgb_model
    }
    
    best_score = 0
    best_model_name = None
    model_scores = {}
    
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_val_selected)
        accuracy = accuracy_score(y_val, y_pred)
        model_scores[name] = accuracy
        print(f"{name} Accuracy: {accuracy:.4f}")
        
        if accuracy > best_score:
            best_score = accuracy
            best_model_name = name
    
    print(f"\nBest single model: {best_model_name} with accuracy {best_score:.4f}")
    
    # Voting Ensemble with weights based on model performance
    # This can give better models more influence
    weights = [model_scores[name] for name in ['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM']]
    
    print("\nTraining Voting Ensemble...")
    voting_clf = VotingClassifier(
        estimators=[
            ('rf', rf),
            ('gb', gb),
            ('xgb', xgb_model),
            ('lgb', lgb_model)
        ],
        voting='soft',
        weights=weights  # Weight models by their validation accuracy
    )
    
    # Combine feature selection with voting classifier in a pipeline
    ensemble_pipeline = Pipeline([
        ('feature_selection', selector),
        ('ensemble', voting_clf)
    ])
    
    # Train final model on full training data
    print("Training final ensemble model on full dataset...")
    ensemble_pipeline.fit(X, y)
    
    # Save model for future use
    joblib.dump(ensemble_pipeline, 'salary_predictor.joblib')
    print("Model saved as 'salary_predictor.joblib'")
    
    return ensemble_pipeline

if __name__ == "__main__":
    main()

Loading and preparing data...
Training data shape: (1280, 317)
Testing data shape: (854, 316)
Processed training data shape: (1280, 90)
Processed test data shape: (854, 93)
Training multiple models...
Performing feature selection...
Selected 36 features out of 90
Training Random Forest...
Random Forest Accuracy: 0.7539
Training Gradient Boosting...
Gradient Boosting Accuracy: 0.7656
Training XGBoost...
XGBoost Accuracy: 0.7773
Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000869 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6819
[LightGBM] [Info] Number of data points in the train set: 1024, number of used features: 36
[LightGBM] [Info] Start training from score -0.937510
[LightGBM] [Info] Start training from score -1.117341
[LightGBM] [Info] Start training from score -1.268511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

  File "c:\Users\damod\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# Enhanced Model Training Report

## Dataset Overview
- Training Data Shape: (1280, 317)
- Testing Data Shape: (854, 316)

---

## Exploratory Data Analysis

### Target Distribution
salary_category
- high: 501
- low: 419
- medium: 360

### Target Distribution by `job_state`
| Job State | High | Low | Medium |
|:---------|-----:|----:|-------:|
| ak | 0.00 | 0.00 | 1.00 |
| al | 0.00 | 1.00 | 0.00 |
| ar | 0.00 | 1.00 | 0.00 |
| az | 0.00 | 0.50 | 0.50 |
| ca | 0.40 | 0.42 | 0.18 |
| co | 0.38 | 0.12 | 0.50 |
| ct | 1.00 | 0.00 | 0.00 |
| dc | 0.25 | 0.12 | 0.62 |
| fl | 0.00 | 0.50 | 0.50 |
| ga | 0.50 | 0.40 | 0.10 |
| ... | ... | ... | ... |

---

### Target Distribution by `feature_1`
| Feature 1 | High | Low | Medium |
|:----------|-----:|----:|-------:|
| a         | 0.41 | 0.30 | 0.29 |
| b         | 0.00 | 1.00 | 0.00 |
| c         | 0.00 | 1.00 | 0.00 |
| d         | 0.20 | 0.60 | 0.20 |
| e         | 0.00 | 1.00 | 0.00 |

---

## Feature Correlations

### `feature_3`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.37 | 0.35 | 0.28 |
| true  | 0.55 | 0.14 | 0.31 |

### `feature_4`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.39 | 0.34 | 0.27 |
| true  | 0.36 | 0.10 | 0.54 |

### `feature_5`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.37 | 0.37 | 0.26 |
| true  | 0.48 | 0.13 | 0.38 |

### `feature_6`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.35 | 0.39 | 0.26 |
| true  | 0.41 | 0.30 | 0.29 |

### `feature_7`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.39 | 0.34 | 0.26 |
| true  | 0.39 | 0.31 | 0.31 |

### `feature_8`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.45 | 0.26 | 0.29 |
| true  | 0.12 | 0.63 | 0.24 |

---

### `feature_10` (by Value)
| Feature 10 | High | Low | Medium |
|:----------|-----:|----:|-------:|
| 6.0   | 0.00 | 1.00 | 0.00 |
| 12.0  | 0.24 | 0.34 | 0.42 |
| 24.0  | 0.29 | 0.34 | 0.37 |
| 180.0 | 0.50 | 0.33 | 0.17 |
| 300.0 | 0.00 | 0.00 | 1.00 |

---

### `feature_11`
| Value | High | Low | Medium |
|:------|-----:|----:|-------:|
| false | 0.14 | 0.58 | 0.28 |
| true  | 0.40 | 0.32 | 0.28 |

---

⚠ Warning:
- Error in `feature_9` binning: numpy boolean subtract not supported, use bitwise_xor (^) or logical_xor.

---

## Processed Dataset
- Processed Training Data Shape: (1280, 176)
- Processed Testing Data Shape: (854, 182)

### Encoded Target Classes
- 0 -> high (501)
- 1 -> low (419)
- 2 -> medium (360)

---

## Model Training and Evaluation

### RandomForest
- CV Score: 0.7414
- Validation Score: 0.7461
- Training Time: 4.07s

### XGBoost
- CV Score: 0.7453
- Validation Score: 0.7617
- Training Time: 14.41s

### LightGBM
- CV Score: 0.7414
- Validation Score: 0.7500
- Training Time: 14.20s
- Training Logs:
  - Auto-choosing col-wise multi-threading
  - Total bins ~12800
  - Positive gain warning ignored.

### CatBoost
- CV Score: 0.7477
- Validation Score: 0.7227
- Training Time: 119.01s

### GradientBoosting
- CV Score: 0.7477
- Validation Score: 0.7773
- Training Time: 193.42s

---

✅ Best Base Model:
- GradientBoosting (Validation Score: 0.7773)

---

## Ensemble Model (Voting)
- Validation Score: 0.7734

---

# 📈 Final Summary
- Best Base Model: GradientBoosting
- Ensemble Model (Voting): Validation Score 0.7734


In [1]:
import joblib
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the model
model = joblib.load('salary_predictor.joblib')

# Load test data
test_df = pd.read_csv('test.csv')

# Load training data to get category values and for label encoder
train_df = pd.read_csv('train.csv')

# Get all possible state and feature_1 values from both datasets
all_states = set(train_df['job_state'].dropna().unique()).union(set(test_df['job_state'].dropna().unique()))
all_feature1 = set(train_df['feature_1'].dropna().unique()).union(set(test_df['feature_1'].dropna().unique()))

# Load the feature columns that were used during training
categorical_cols = joblib.load('feature_columns.joblib')

# Import the preprocessing function
# Since we can't import directly from a notebook, we need to define it here
# or use the function from your saved model.py file
def preprocess_data(df, all_states, all_feature1, is_training=False, categorical_cols=None):
    # Define feature groups
    categorical_features = ['job_title', 'job_state', 'feature_1']
    boolean_features = ['feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_10', 'feature_11']
    numerical_features = ['feature_2', 'feature_9', 'feature_12'] + [f'job_desc_{str(i).zfill(3)}' for i in range(1, 301)]
    date_feature = ['job_posted_date']
    
    # Make a copy of the dataframe to avoid modifying the original
    data = df.copy()
    
    # Store the target if it exists (not for test data)
    if 'salary_category' in data.columns and is_training:
        target = data['salary_category']
        # Label encode target for model training
        le = LabelEncoder()
        y = le.fit_transform(target)
    else:
        y = None

    # Handle missing values in numerical features
    for col in numerical_features:
        if col in data.columns:
            if data[col].dtype == 'object':
                data[col] = data[col].replace('', np.nan)
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Convert boolean features to 0/1
    for col in boolean_features:
        if col in data.columns:
            data[col] = data[col].fillna(False).astype(int)
    
    # Handle job_title
    if 'job_title' in data.columns:
        data['job_title'] = data['job_title'].fillna('Unknown')
        
        # Create new feature based on job title patterns
        data['is_senior'] = data['job_title'].str.lower().str.contains('senior|sr|lead|principal').fillna(False).astype(int)
        data['is_junior'] = data['job_title'].str.lower().str.contains('junior|jr|associate|entry').fillna(False).astype(int)
        data['is_developer'] = data['job_title'].str.lower().str.contains('develop|programmer|coder|engineer').fillna(False).astype(int)
        data['is_specialist'] = data['job_title'].str.lower().str.contains('special|expert|consult').fillna(False).astype(int)
        
        # Group rare job titles
        title_counts = data['job_title'].value_counts()
        rare_titles = title_counts[title_counts < 10].index
        data['job_title'] = data['job_title'].apply(lambda x: 'Other_Title' if x in rare_titles else x)
        
        if is_training:
            # Save encoder for test data
            job_encoder = TargetEncoder()
            data['job_title_encoded'] = job_encoder.fit_transform(data['job_title'], target)
            joblib.dump(job_encoder, 'job_title_encoder.joblib')
        else:
            # Load encoder for test data
            if os.path.exists('job_title_encoder.joblib'):
                job_encoder = joblib.load('job_title_encoder.joblib')
                data['job_title_encoded'] = job_encoder.transform(data['job_title'])
            else:
                # If no encoder found, use mean encoding as fallback
                data['job_title_encoded'] = 0.5  # Default value
    
    # Handle state and feature_1 - ensure consistent one-hot encoding across train and test
    for col in ['job_state', 'feature_1']:
        if col in data.columns:
            data[col] = data[col].fillna('Unknown')
    
    # Process date feature
    if 'job_posted_date' in data.columns:
        data['job_posted_date'] = data['job_posted_date'].fillna('2000/01')
        
        # Extract year and month safely
        def extract_year(date_str):
            try:
                return int(date_str[:4])
            except (TypeError, ValueError, IndexError):
                return 2000
        
        def extract_month(date_str):
            try:
                return int(date_str.split('/')[1])
            except (TypeError, ValueError, IndexError):
                return 1
        
        data['job_posted_year'] = data['job_posted_date'].apply(extract_year)
        data['job_posted_month'] = data['job_posted_date'].apply(extract_month)
        
        # Create cyclical features for month to capture seasonality
        data['month_sin'] = np.sin(2 * np.pi * data['job_posted_month']/12)
        data['month_cos'] = np.cos(2 * np.pi * data['job_posted_month']/12)
        
        # Create feature for recency
        data['job_recency'] = data['job_posted_year'] * 12 + data['job_posted_month']
        
        # Handle year ranges
        mean_year = 2022
        data['job_posted_year_norm'] = data['job_posted_year'] - mean_year
    
    # Feature engineering for feature_9 and feature_2
    if 'feature_9' in data.columns:
        median_val = data['feature_9'].median() if not pd.isna(data['feature_9'].median()) else 0
        data['feature_9'] = data['feature_9'].fillna(median_val)
        
        try:
            data['feature_9_bin'] = pd.qcut(data['feature_9'].rank(method='first'), 
                                           q=5, 
                                           labels=[0, 1, 2, 3, 4]).astype(int)
        except ValueError:
            data['feature_9_bin'] = pd.cut(data['feature_9'], 
                                          bins=5, 
                                          labels=[0, 1, 2, 3, 4],
                                          include_lowest=True).astype(int)
        
        if 'feature_2' in data.columns:
            data['feature_2_9_interaction'] = data['feature_2'] * data['feature_9']
    
    if 'feature_2' in data.columns:
        data['feature_2'] = data['feature_2'].fillna(data['feature_2'].median())
        data['feature_2_squared'] = data['feature_2'] ** 2
        data['feature_2_sqrt'] = np.sqrt(np.abs(data['feature_2']))
        try:
            data['feature_2_bin'] = pd.qcut(data['feature_2'].rank(method='first'), 
                                           q=5, 
                                           labels=[0, 1, 2, 3, 4]).astype(int)
        except ValueError:
            data['feature_2_bin'] = pd.cut(data['feature_2'], 
                                          bins=5, 
                                          labels=[0, 1, 2, 3, 4],
                                          include_lowest=True).astype(int)
    
    # Sum of boolean features
    boolean_cols = [col for col in boolean_features if col in data.columns]
    if boolean_cols:
        data['boolean_sum'] = data[boolean_cols].sum(axis=1)
        data['boolean_sum_squared'] = data['boolean_sum'] ** 2
    else:
        data['boolean_sum'] = 0
        data['boolean_sum_squared'] = 0
    
    # Feature interaction
    if 'feature_10' in data.columns and 'feature_8' in data.columns:
        data['feature_10_8_interaction'] = data['feature_10'] * data['feature_8']
    
    # Job desc features
    job_desc_cols = [f'job_desc_{str(i).zfill(3)}' for i in range(1, 301) if f'job_desc_{str(i).zfill(3)}' in data.columns]
    if job_desc_cols:
        # Fill NAs with 0
        data[job_desc_cols] = data[job_desc_cols].fillna(0)
        
        # Create aggregate features
        data['job_desc_mean'] = data[job_desc_cols].mean(axis=1)
        data['job_desc_std'] = data[job_desc_cols].std(axis=1)
        data['job_desc_min'] = data[job_desc_cols].min(axis=1)
        data['job_desc_max'] = data[job_desc_cols].max(axis=1)
        data['job_desc_sum'] = data[job_desc_cols].sum(axis=1)
        
        data['job_desc_q25'] = data[job_desc_cols].quantile(0.25, axis=1)
        data['job_desc_q75'] = data[job_desc_cols].quantile(0.75, axis=1)
        data['job_desc_iqr'] = data['job_desc_q75'] - data['job_desc_q25']
        
        # PCA features
        if len(job_desc_cols) > 10:
            if os.path.exists('job_desc_pca.joblib'):
                pca = joblib.load('job_desc_pca.joblib')
                job_desc_pca = pca.transform(data[job_desc_cols])
                
                # Add PCA features
                for i in range(min(15, job_desc_pca.shape[1])):
                    data[f'job_desc_pca_{i}'] = job_desc_pca[:, i]
    
    # Manual one-hot encoding for job_state and feature_1
    for state in all_states:
        data[f'state_{state}'] = (data['job_state'] == state).astype(int)
    
    for feat in all_feature1:
        data[f'feat1_{feat}'] = (data['feature_1'] == feat).astype(int)

    # Drop original columns after encoding
    data = data.drop(['job_state', 'feature_1'], axis=1, errors='ignore')
    
    # Create feature set for modeling
    columns_to_exclude = ['obs', 'job_title', 'salary_category', 'job_posted_date'] + job_desc_cols
    feature_cols = [col for col in data.columns if col not in columns_to_exclude]
    
    # Use only the columns from training
    X = data[feature_cols]
    
    # Ensure all training columns exist in test data
    if categorical_cols is not None:
        for col in categorical_cols:
            if col not in X.columns:
                X[col] = 0
        
        # Make sure columns are in the same order
        X = X[categorical_cols]
    
    return X, y, feature_cols

# Process test data using stored values
X_test, _, _ = preprocess_data(test_df, all_states, all_feature1, is_training=False, categorical_cols=categorical_cols)

# Since we're using a pipeline with feature selection, we can predict directly
predictions = model.predict(X_test)

# Set up label encoder
le = LabelEncoder()
le.fit(train_df['salary_category'])

# Map predictions to labels
predictions_labels = le.inverse_transform(predictions)

# Create submission file
submission = pd.DataFrame({
    'obs': test_df['obs'],
    'salary_category': predictions_labels
})

# Ensure the output directory exists
os.makedirs('data', exist_ok=True)

# Save predictions
submission.to_csv('solution_format.csv', index=False)
print(f"Predictions saved to solution_format.csv")
print(f"Prediction distribution: {pd.Series(predictions_labels).value_counts().to_dict()}")

C:\Users\damod\AppData\Local\Temp\ipykernel_16404\2570537740.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['is_senior'] = data['job_title'].str.lower().str.contains('senior|sr|lead|principal').fillna(False).astype(int)
C:\Users\damod\AppData\Local\Temp\ipykernel_16404\2570537740.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['is_junior'] = data['job_title'].str.lower().str.contains('junior|jr|associate|entry').fillna(False).astype(int)
C:\Users\damod\AppData\Local\Temp\ipykernel_16404\2570537740.py:64: Per

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- feature_10_8_diff
- feature_10_8_sum
- feature_10_feature_11_or
- feature_2_9_ratio
- feature_2_cubed
- ...
